### Multimodal Classification: text + tabular + image
### Early fusion NN + Feature fusion with XGBoost and SVM

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, Bidirectional, LSTM,
    Dense, Dropout, Concatenate, GlobalAveragePooling1D
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.preprocessing import image as keras_image

In [ ]:
"""my plan is to first perform early fusion between the tabular features and the
 text data (using tokenized text as input). This combined representation will be fed 
 into a suitable neural network. In parallel, the image data will be processed using a 
 pretrained neural network to extract visual features. Finally, I will apply feature-level 
 fusion between the learned representations from the text–tabular model and the image model, and use the 
 fused features as input to XGBoost and SVM classifiers for final prediction."""

'my plan is to first perform early fusion between the tabular features and the\n text data (using tokenized text as input). This combined representation will be fed into a suitable neural network. In parallel, the image data will be processed using a pretrained neural network to extract visual features. Finally, I will apply feature-level fusion between the learned representations from the text–tabular model and the image model, and use the fused features as input to XGBoost and SVM classifiers for final prediction.'

In [5]:
df = pd.read_csv("FoodFactsCleaned.csv")

In [6]:
TEXT_COLS = [
    "brand_cleaned",
    "allergens_cleaned",
    "ingredients_text_cleaned",
    "countries_cleaned",
    "additives_cleaned",
]

TABULAR_COLS = [
    'nova_group', 'fat_100g',
    'saturated_fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
    'proteins_100g', 'contains_palm_oil', 'vegetarian_status', 'vegan_status',
    'nutrient_level_fat', 'nutrient_level_saturated_fat',
    'nutrient_level_sugars', 'nutrient_level_salt', 'ecoscore_grade', 'ecoscore_score',
    'carbon_footprint_100g', 'additives_count', 'sugar_ratio',
    'energy_density', 'protein_ratio', 'macro_balance', 'healthy_score',
    'log_energy_kcal_100g', 'log_salt_100g'
]

TARGET_COL = "nutriscore_letter"         
IMAGE_PATH_COL = "image_160_path"  
RANDOM_STATE = 42
TEST_SIZE = 0.2

# Text tokenization
MAX_WORDS = 30000
MAX_LEN = 200

In [7]:
for c in TEXT_COLS:
    df[c] = df[c].fillna("").astype(str)

df["text_concat"] = df[TEXT_COLS].agg(" ".join, axis=1)

print("Rows after image path filtering:", len(df))

Rows after image path filtering: 5138
